## Imports

In [ ]:
import pandas as pd

from utils import parse_text

## Constants

In [ ]:
# Original file prepared by our anno team
ANNOTATED_SENSITIVE_DATA = "data/annotated-sensitive-en.csv"

# Below files need to be get from prod and stage databases (via Metabase or Ackee DB),
# since not all the articles, in the document provided by anno team (see above), have perex and body.
# See below for the SQL commands to get these.
CMS_PROD_SENSITIVE_DATA = "data/cms-sensitive-articles-prod.csv"
CMS_STAGE_SENSITIVE_DATA = "data/cms-sensitive-articles-stage.csv"
CMS_NONSENSITIVE_DATA = "data/cms-nonsensitive-articles-prod-all.csv"

# Processed (output) files
PROCESSED_SENSITIVE_DATA = "data/processed-sensitive-data.csv"
PROCESSED_NONSENSITIVE_DATA = "data/processed-nonsensitive-data-all.csv"

## Get Data

In [ ]:
df = pd.read_csv(ANNOTATED_SENSITIVE_DATA)

In [ ]:
df["is_stage"] = df.id.apply(lambda x: "stage" in str(x))
df.head(5)

In [ ]:
df["id_clean"] = df.id.apply(lambda x: str(x).split("/")[-1])
df.head(5)

In [ ]:
df = df[df["id_clean"] != "nan"]
df.reset_index(drop=True, inplace=True)
df.head(5)

In [ ]:
print(f"Number of sensitive articles: {len(df)}")

In [ ]:
assert len(df) == len(set(list(df.id))), f"Some articles are probably duplicates {len(df)} vs. {len(set(list(df.id)))}"

In [ ]:
",".join([f"'{i}'" for i in list(set(df.id_clean.values))])

## Sensitive articles need to be extracted from both stage and production

### SQL query for metabase to select sensitive articles

SELECT id, title, perex, body<br>
FROM article<br> 
WHERE language = 'en' and id in ('119792326','120286189','120376048','123306778','120007036','126175721','120536534','119676988','119092035','120623528','120485832','119074320','120383310','126923797','119002865','118987848','119432400','118965701','119533208','127135586','119973264','120413981','122894877','123293435','125763068','125467798','123459623','119051746','118793886','119567189','119570755','126382692','119161377','123288785','121602148','115014804','120605445','118289613','117085549','120415122','119181572','123802005','119003668','119581215','120210500','118970748','118986285','119051490','119000800','120842062','122263268','120656461','119220655','119773732','119116768','120019036','127067097','119427330','120716623','119326243','119041383','120576334','123901772','119679526','118965833','119617079','120578554','117582150','119389524','118970406','119048083','119003195','119962172','118965891','117631761','119476570','118965817','118963471','118960774','122905229','119240456','119969327','122809825','117329444','119051512','118227909','124213597','118869232','120556411','118945045','119753246','120158510','120549278','125800532','118965743','119035325','117045375','120287155','119163893','119248932','120671844','119723016','119004795','120378128','119770559','120433513','120525659','119328342','115306691','119130335','125824341','119303926','119162334','118960445','120517115','118959110','119128766','120100896','119831094','115051227','120390313','117135957','120244689','119121032','117264952','125031067','120548774','120257688','119099849','119460895','124481682','120311592','118821301','118965632','120052128','119696106','119435321','121616216','120309517','119003313','115904657','120139590','119235066','119049105','125891543','119220804','118927859','118691995','124637319','125808037','119975811','119464167','120549562','118965628','127011360','120523475','119756915','127171104','121221160','120828952','118965101','124307777','118965807','119811942','119823666','119734671','120550673','119318267','120492319','118967286','119053857','120538202','122758776','119198858','120062749','126471723','118197910','119635402','120438694','120433514','126771836','118950574','119051989','119331855','120337965','120650142','119305479','120854937','118987202','123810031','122391313','118961657','120112189','127150381','118900887','119324327','124507605','118835002','119136422')

### SQL query for metabase to select nonsensitive articles

SELECT id, title, perex, body<br>
FROM article<br>
WHERE language = 'en' and title != '' and perex != '' and id not in ('119792326','120286189','120376048','123306778','120007036','126175721','120536534','119676988','119092035','120623528','120485832','119074320','120383310','126923797','119002865','118987848','119432400','118965701','119533208','127135586','119973264','120413981','122894877','123293435','125763068','125467798','123459623','119051746','118793886','119567189','119570755','126382692','119161377','123288785','121602148','115014804','120605445','118289613','117085549','120415122','119181572','123802005','119003668','119581215','120210500','118970748','118986285','119051490','119000800','120842062','122263268','120656461','119220655','119773732','119116768','120019036','127067097','119427330','120716623','119326243','119041383','120576334','123901772','119679526','118965833','119617079','120578554','117582150','119389524','118970406','119048083','119003195','119962172','118965891','117631761','119476570','118965817','118963471','118960774','122905229','119240456','119969327','122809825','117329444','119051512','118227909','124213597','118869232','120556411','118945045','119753246','120158510','120549278','125800532','118965743','119035325','117045375','120287155','119163893','119248932','120671844','119723016','119004795','120378128','119770559','120433513','120525659','119328342','115306691','119130335','125824341','119303926','119162334','118960445','120517115','118959110','119128766','120100896','119831094','115051227','120390313','117135957','120244689','119121032','117264952','125031067','120548774','120257688','119099849','119460895','124481682','120311592','118821301','118965632','120052128','119696106','119435321','121616216','120309517','119003313','115904657','120139590','119235066','119049105','125891543','119220804','118927859','118691995','124637319','125808037','119975811','119464167','120549562','118965628','127011360','120523475','119756915','127171104','121221160','120828952','118965101','124307777','118965807','119811942','119823666','119734671','120550673','119318267','120492319','118967286','119053857','120538202','122758776','119198858','120062749','126471723','118197910','119635402','120438694','120433514','126771836','118950574','119051989','119331855','120337965','120650142','119305479','120854937','118987202','123810031','122391313','118961657','120112189','127150381','118900887','119324327','124507605','118835002','119136422')
LIMIT 100000


## Link sensitive article id with its content

In [ ]:
df_prod_articles_sensitive = pd.read_csv(CMS_PROD_SENSITIVE_DATA, delimiter=",", encoding="utf-8")
df_stage_articles_sensitive = pd.read_csv(CMS_STAGE_SENSITIVE_DATA, delimiter=",", encoding="utf-8")

In [ ]:
df_prod_articles_sensitive.columns

In [ ]:
len(set(df_prod_articles_sensitive.id) & set(df_stage_articles_sensitive.id))

In [ ]:
df_prod_articles_sensitive = df_prod_articles_sensitive[["id", "title", "perex", "body"]]
df_stage_articles_sensitive = df_stage_articles_sensitive[["id", "title", "perex", "body"]]

In [ ]:
def get_data(row):
    return {
        "title": parse_text(row.title.values[0]),
        "perex": parse_text(row.perex.values[0]),
        "body": parse_text(row.body.values[0]),
    }

In [ ]:
sensitive_data = {}
prod_ids = list(df_prod_articles_sensitive.id.astype(str))
stage_ids = list(df_stage_articles_sensitive.id.astype(str))
for _, row in df.iterrows():
    if row.is_stage:
        sensitive_data[str(row.id_clean)] = get_data(
            df_stage_articles_sensitive[df_stage_articles_sensitive.id == int(row.id_clean)]
        )
    else:
        sensitive_data[str(row.id_clean)] = get_data(
            df_prod_articles_sensitive[df_prod_articles_sensitive.id == int(row.id_clean)]
        )

In [ ]:
print(f"Number of sensitive articles: {len(sensitive_data)}")

In [ ]:
df_sensitive = pd.DataFrame.from_dict(sensitive_data, orient="index")
df_sensitive.index.name = "id"
df_sensitive.head(5)

In [ ]:
df_sensitive.to_csv(PROCESSED_SENSITIVE_DATA)

## Link non-sensitive article id with its content

In [ ]:
df_prod_articles = pd.read_csv(CMS_NONSENSITIVE_DATA, delimiter=",", encoding="utf-8")

In [ ]:
df_prod_articles["body"] = df_prod_articles["body"].apply(lambda x: parse_text(str(x)))

In [ ]:
df_prod_articles.set_index("id").to_csv(PROCESSED_NONSENSITIVE_DATA)